In [1]:
import MySQLdb
import requests
import json

db = MySQLdb.connect(
    host="172.99.106.245",
    user="root",
    passwd="K0llaP4ss",
    db="poi2")
cur = db.cursor()

In [2]:
cur.execute("select * from geo_tags limit 10")
for s in cur.fetchall():
    print s

(4018983L, 36264314L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4123619L, 35155320L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4252467L, 33948800L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4266388L, 35488984L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4293159L, 36264368L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4347073L, 36390050L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4356330L, 35182537L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4365610L, 36244158L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4365840L, 37083114L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)
(4365861L, 33990160L, 'earth', 1, Decimal('0E-8'), Decimal('0E-8'), None, None, None, None, None)


In [3]:
or_w = (42.8470881, -124.7595306)
or_s = (41.993219, -121.259210)
or_e = (45.595339, -116.409837)
or_n = (46.306281, -124.011349)
or_coords = {
    'or_w': or_w[1],
    'or_e': or_e[1],
    'or_s': or_s[0],
    'or_n': or_n[0],
}
print or_coords
query = "select count(*) from geo_tags where gt_lon < {or_e} and gt_lon > {or_w} and gt_lat > {or_s} and gt_lat < {or_n} and gt_region='OR'".format(**or_coords)
print(query)
cur.execute(query)
or_poi = cur.fetchall()
print or_poi[0]


{'or_s': 41.993219, 'or_n': 46.306281, 'or_w': -124.7595306, 'or_e': -116.409837}
select count(*) from geo_tags where gt_lon < -116.409837 and gt_lon > -124.7595306 and gt_lat > 41.993219 and gt_lat < 46.306281 and gt_region='OR'
(10725L,)


In [4]:
cur.execute("select * from geo_tags where gt_page_id=77156")
print cur.fetchall()

((268338307L, 77156L, 'earth', 1, Decimal('45.41956000'), Decimal('-122.66755000'), 10000L, 'city', None, 'US', 'OR'),)


In [5]:
payload = {'pageids': '77156', 'format': 'json', 'action': 'query', 'prop': 'extracts'}

import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def req(p):
  r = requests.get('https://en.wikipedia.org/w/api.php', params=p)
  print(TAG_RE.sub('', json.loads(r.text)['query']['pages'][p['pageids']]['extract']))

print req(payload)
    

Lake Oswego /ɒsˈwiːɡoʊ/ is a city in the State of Oregon, primarily in Clackamas County with small portions extending into neighboring Multnomah and Washington counties. Located south of Portland surrounding the 405-acre (1.64 km2) Oswego Lake, the town was founded in 1847 and incorporated as Oswego in 1910. The city was the hub of Oregon's brief iron industry in the late 19th century and is today an affluent suburb of Portland. As of the 2010 census, the city had a total population of 36,619, a 3.8% increase over the 2000 population of 35,278.



History
Early history
The Clackamas Indians once occupied the land that later became Lake Oswego, but diseases transmitted by European explorers and traders killed most of the natives. Before the influx of non-native people via the Oregon Trail, the area between the Willamette River and Tualatin River had a scattering of early pioneer homesteads and farms.
19th century

As settlers arrived, encouraged by the Donation Land Claim Act of 1850 an

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)
import pandas as pd

print twenty_train.target_names

df = pd.DataFrame(twenty_train.data)
df.head()


['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


0
0  From: sd345@city.ac.uk (Michael Collier)\nSubj...
1  From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\...
2  From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...
3  From: s0612596@let.rug.nl (M.M. Zwart)\nSubjec...
4  From: stanly@grok11.columbiasc.ncr.com (stanly...

In [7]:
cur.execute("select body from locations")
locs = cur.fetchall()
locs = [l[0].decode('utf-8', 'replace').strip() for l in locs]
cur.execute("select count(*) from locations")
cur.fetchall()

((3478L,),)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(locs)
print X_train_counts.shape

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape


(3478, 38441)


(3478, 38441)

In [14]:
cur.execute("select * from locations where body like '%forest%' limit 100")
forests = cur.fetchall()
t = []
fors = [f[1] for f in forests]

for l in locs:
    if l in fors:
        t.append(1)
    else:
        t.append(0)

filter

/home/ubuntu/venv/lib/python2.7/site-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
from sklearn.naive_bayes import MultinomialNB

def refit_model(data, targets):
    clf = MultinomialNB().fit(X_train_tfidf, targets)
    return clf


    